# Math 4 - Linear Algebra: The Inverse of a Matrix

In [1]:
import numpy as np
import scipy.linalg

his notebook is part of a series of math topics. You can find the previous chapter **Linear Algebra: Gaussian Elimination** [here](https://github.com/jaunerc/cs-math/blob/master/math-3-linear-algebra-gaussian-elimination.ipynb)

## 1. The inverse of a matrix
Let's start with a simple case. The (multiplicative) inverse of every real number $r$ (except zero) is just $\frac{1}{r}$ or in a different notation $r^{-1}$. This is because the product of a real number with its inverse must be equal to $1$. So we have $r^{-1} * r = \frac{1}{r} * r = 1$.

If we now multiply a matrix with its inverse we got the identity matrix $E$ instead of $1$. The equation is $A * A^{-1} = E$. Furthermore, not every matrix has an inverse. Therefore, finding the inverse can be hard. Luckily numpy has a function `inv` for this task.

The example below calculates the inverse of the matrix $A$. The result is then verified such that $A * A^{-1} = E$ holds. The function `allclose` compares two arrays element-wise and returns True if they are equal within a tolerance.

In [7]:
A = np.linspace([1, 3], 6, num=2)
A_inv = np.linalg.inv(A)

np.allclose(A.dot(A_inv), np.eye(2)) # Verifying the result

True

The `inv` function throws an error if the given matrix is not invertible. For example this is the case for every singular matrix.

In [8]:
B = np.array([[2, 2], [2,2]])
np.linalg.inv(B)

LinAlgError: Singular matrix

## 2. Find the inverse by yourself
But how do we find the inverse whitout a predefined function? For square-matrices with 2 or 3 dimensions exists easy equations to find the inverse thanks to [Cramer's rule](https://en.wikipedia.org/wiki/Cramer%27s_rule). Actually also for higher dimensions but then we have huge equations.

In the 2x2 case we find the inverse as follows.

$$A^{-1} = \begin{bmatrix}a & b \\ c & d \end{bmatrix}^{-1} = \frac{1}{det(A)} \begin{bmatrix}d & -b \\ -c & a \end{bmatrix}$$

To calculate this we need the determinant of the matrix. For 2x2 matrices this is just

$$det(A) = ad - bc$$

As we can see in the equation the determinant has to be nonzero. Otherwise we have a division by zero. Therefore a matrix is invertible if the determinant is not zero.

_Note: The test with the determinat only works if the matrix elements are within a field. This is the case in every example in this notebook._

The following snippet shows the calculation to find the inverse for a 2x2 matrix. The result is the same as the result from the `inv` function.

In [15]:
A_det = np.linalg.det(A)
A_prepared = np.array([[A[1][1], -A[0][1]], [-A[1][0], A[0][0]]])
A_own_inv = A_prepared.dot(1 / A_det)

np.allclose(np.linalg.inv(A), A_own_inv)

True

### 2.1 Gauss-Jordan Method
To find the inverse of a $n \times n$ matrix we can use an adaption of the Gaussian elimination. The matrix we want to invert is extended with the identity matrix of the same shape. The resulting augmented matrix $[A|I_3]$ is then transformed to the reduced row echelon form.

$$
A = \begin{bmatrix}1 & 2 & 7 \\ 5 & 8 & 3 \\ 1 & 9 & 3 \end{bmatrix}, I_3 = \begin{bmatrix}1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{bmatrix}
$$

The augmented matrix

$$
[A|I_3] = \begin{bmatrix}1 & 2 & 7 & 1 & 0 & 0 \\ 5 & 8 & 3 & 0 & 1 & 0 \\ 1 & 9 & 3 & 0 & 0 & 1 \end{bmatrix}
$$

And the reduced augmented matrix

$$
[I_3|A^{-1}] = \begin{bmatrix}1 & 0 & 0 & -0.01293 & 0.24568 & -0.21551 \\
0 & 1 & 0 & -0.05172 & -0.01724 & 0.13793 \\
0 & 0 & 1 & 0.15948 & -0.03017 & -0.00862 \end{bmatrix}
$$

As you can see the reduced matrix is separated into two parts. The first three columns left are the identity matrix again and the other columns are the inverse of $A$.

To implement this method in code we need the `rref` function again.

In [16]:
def rref(Mat):
    # Calculates the row-reduced-echelon-form for the given matrix
    # from https://rosettacode.org/wiki/Reduced_row_echelon_form
    M = np.copy(Mat)
    lead = 0
    row_count = len(M)
    col_count = len(M[0])
    for r in range(row_count):
        if col_count <= lead:
            return M
        i = r
        while M[i][lead] == 0:
            i += 1
            if row_count == i:
                i = r
                lead += 1
                if col_count == lead:
                    return M
        M[i], M[r] = M[r], M[i]
        if not M[r][lead] == 0:
            M[r] = M[r] / M[r][lead]
        for i in range(row_count):
            if not i == r:
                lv = M[i][lead]
                M[i] = [iv - lv*rv for rv,iv in zip(M[r], M[i])]
        lead += 1
    return M

For example we define the same 3x3 matrix as above.

In [18]:
A = np.array([[1, 2, 7], [5, 8, 3], [1, 9, 3]])
A

array([[1, 2, 7],
       [5, 8, 3],
       [1, 9, 3]])

And build the **augmented matrix** with the original matrix `A` and the identity matrix. Numpy has a function `eye` to create a identity matrix of a specific size.

In [19]:
dim_A = len(A[0])
augmented = np.concatenate((A, np.eye(dim_A)), axis=1)
augmented

array([[1., 2., 7., 1., 0., 0.],
       [5., 8., 3., 0., 1., 0.],
       [1., 9., 3., 0., 0., 1.]])

The next step is reducing the augmented matrix to the **rref**.

In [20]:
rref_augmented = rref(augmented)
rref_augmented

array([[ 1.        ,  0.        ,  0.        , -0.01293103,  0.24568966,
        -0.21551724],
       [ 0.        ,  1.        ,  0.        , -0.05172414, -0.01724138,
         0.13793103],
       [-0.        , -0.        ,  1.        ,  0.15948276, -0.03017241,
        -0.00862069]])

The inverse of `A` is in the right part of the reduced matrix.

In [21]:
A_inv = rref_augmented[:,3:6]
A_inv

array([[-0.01293103,  0.24568966, -0.21551724],
       [-0.05172414, -0.01724138,  0.13793103],
       [ 0.15948276, -0.03017241, -0.00862069]])

Great! We can verify this with the `inv` function from numpy and it looks good.

In [22]:
np.allclose(A_inv, np.linalg.inv(A))

True